In [179]:
# # Installing necessary dependencies
import sys
# !conda install --yes --prefix {sys.prefix} PIL
# # !{sys.executable} -m pip install ____


# !{sys.executable} -m pip install pytesseract
# !{sys.executable} -m pip install pillow
# !{sys.executable} -m pip install python-poppler
# !{sys.executable} -m pip install opencv-python
# !{sys.executable} -m pip install --upgrade pip
# !conda install -c conda-forge poppler

# !{sys.executable} -m pip install scipy
# !{sys.executable} -m pip install sklearn
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install nltk
# !{sys.executable} -m pip install pandas


In [93]:
import os
from pdf2image import convert_from_path
# import pandas as pd
try:
    from PIL import Image
except ImportError:
    import Image
# import numpy as np
import pytesseract
import os

pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'

In [207]:

# FUnction to extract words from a pdf and save it in as a text file

def pdf_2_txt(path):          
    for file in os.listdir(path): # goes through the folders'
        if file.endswith('.pdf'): # finds pdfs
            p = os.path.join(path,file) # grabs path of pdf
            pdfpages = convert_from_path(p, 500) # Converting each page in the pdf into image file with 500 dpi
            image_counter = 1
            for page in pdfpages:
                filename = f"page_{image_counter}.jpg"
                # Save the image of the page in system
                page.save(filename, 'JPEG')
                image_counter+=1
                text = str(((pytesseract.image_to_string(Image.open(filename))))) # takes image and process into text
                # delete image
                os.remove(filename)
                text = text.replace('-\n', '') # some text processing
                                
                # Checking if the page contains the "Course objective or 
                # description or outline or goals or Learning Outcomes followed by an "end of string" and
                # a new line (As a title)"
                
                if re.match(re.compile("^Course\s(Outline|Objectives|Objective|Description|Goals)$\n"),
                            text) or re.match(re.compile("^Learing Outcomes$\n"), text):
                    filename = filename.replace('.jpg','') # Naming
                    with open(f'{filename}.txt', 'w+') as f: # Opens a text file and writes the characters onto
                        print(text, file=f)
                        print(f'Page_{image_counter}.txt has been saved into {os.getcwd()}')
                    break # stops iteration as soon as page is found               
                
    return #text
                    
                
        


In [205]:
# Implementing pdf to image conversion
path = os.getcwd()
pdf_2_txt(path)
# pages = convert_from_path(path, 500) 
# for page in pages:
#     page.save('test.jpg')

Page_3.txt has been saved into C:\Users\15713\Desktop


In [366]:
# Below, I try to do some text processing and tokenization to see word frequencies

import codecs
import re
import copy
import collections

import numpy as np
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from __future__ import division
import matplotlib



In [51]:
nltk.download('stopwords') # downloads all stopwords to avoid

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\15713\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [52]:
from nltk.corpus import stopwords 

In [176]:
# In case we have several .txt files of the same pdf, then we use this function to combine into 1 text file (not saved)
def combine_pages(path):
    # Combining pages
    pages = ''
    for file in os.listdir(path):
        if file.endswith('.txt'):
            with codecs.open(file, 'r') as f: 
                page = f.read()
                pages+=page
    return pages

In [180]:
# getting a list of unecessary words like 'the', 'or',... to avoid
avoid = stopwords.words('english')
avoid.append('edu') # adding our own word to the list

In [336]:
# Function to tokenize a text file into sentences or phrases
def get_sentence_counter(path=None, text=None):
    if path:
        path = os.getcwd()
        for file in os.listdir():
            if file.endswith('.txt'):
                with codecs.open(file, 'r') as f: 
                    text = f.read()
    else:
        text = text
    # groups all words and white-spaces, then finds words until a punctuation is reached
    regex_of_sentence = re.findall('([\w\s]{0,})[^\w\s]', text) 
    avoid = re.findall('\d', text)
    avoid.append('')
    regex_of_sentence = [x for x in regex_of_sentence if x not in avoid]
    # for i in regex_of_sentence:
    #     print(i)
    return collections.Counter(regex_of_sentence), len(regex_of_sentence)


In [204]:
# Function to tokenize a text file into words
def get_text_counter(text, avoid, word_pattern):
#     tokens = WordPunctTokenizer().tokenize(PorterStemmer().stem(text))
    tokens = WordPunctTokenizer().tokenize(text)
    tokens = list(map(lambda x: x.lower(), tokens)) # converting everything to lower-case
    # cleaning up tokens by getting rid of stopwords and finding word characters
    Puncs = [token for token in tokens if re.match(re.compile("\W"), token)] # finding all punctuations
    Nums = [token for token in tokens if re.match(re.compile("\d"), token)] # finding all digit characters
    # adding punctuations and digits to the list of avoided characters/words
    avoid = Puncs + Nums
    tokens = [token for token in tokens if re.match(word_pattern, token) and token not in avoid] 
    # collections.counter() returns the tokens and their frequency as a dictionary ({freq: token})
    return collections.Counter(tokens), len(tokens) 

In [263]:
# Function to make dataframe for both tokenized words and sentences with their abs and relative freqz
def make_df(counter, size, ind_name):
    abs_freq = np.array([x[1] for x in counter]) # the absolute freqz is found from the collections.counter()
    rel_freq = abs_freq/size # relative freqz is calculated by dividing abs_freqz by the size
    index = [x[0] for x in counter] # the tokens are used as an index 
    # creating the dataframe
    df = pd.DataFrame(data=np.array([abs_freq, rel_freq]).T, 
                      index=index,
                      columns=["Absolute frequency","Relative frequency"])
    # Naming the index title
    df.index.name = ind_name
    return df

In [264]:
# Implementing functions for sentences or phrases
path = os.getcwd()
sentence_count, size = get_sentence_counter(path)
df_sentence = make_df(sentence_count.most_common()[:-n-1:-1], size,
             ind_name='Least Commonly\n Occuring Sentences') # finds the least commonly occuring sentences
df_sentence.head(50)

,Absolute frequency,Relative frequency
Least Commonly Occuring Sentences,,
Your grade will be determined by your participation in discussions and submitted\nreviews,1.0,0.011494
You will be a reviewer or reader for 4 grants and participate in a simulated NIH\nstudy section,1.0,0.011494
\n\nAssignment,1.0,0.011494
NSF and NIH pdfs and web sites,1.0,0.011494
Writing and Reviewing Research Proposals\nReading,1.0,0.011494
\n\n6,1.0,0.011494
quiz on Leedy Ch 11,1.0,0.011494
Leedy Ch11 and completion of the Linked In course on R\n\nEvaluation,1.0,0.011494
Quantitative Analyses and Review of Statistical Procedures\nReading,1.0,0.011494


In [265]:
# Implementing functions for sentences or phrases
path = os.getcwd()
pages = combine_pages(path) # combining every text file in folder
word_count, size = get_text_counter(pages, avoid, word_pattern) # returns tokens freqz and count of tokens
n = 1000 # 1000 of least commonly occuring words
# least commonly occuring words are chosen with the assumption that a syllabus will only mention
# technical terms once or twice while other words like 'students' and so on could be redundant
df_word = make_df(word_count.most_common()[:-n-1:-1], size,
             ind_name='Least Commonly\n Occuring Words') # finds the least commonly occuring words
df_word.head(50)

,Absolute frequency,Relative frequency
Least Commonly Occuring Words,,
proposal,1.0,0.002985
page,1.0,0.002985
submit,1.0,0.002985
reviews,1.0,0.002985
submitted,1.0,0.002985
discussions,1.0,0.002985
participation,1.0,0.002985
determined,1.0,0.002985
grade,1.0,0.002985


In [364]:
# Just showing a complete list of the words tokenized
# uncomment if you want to try
# n = 1000
# print([x[0] for x in word_count.most_common()[:-n-1:-1]])

In [363]:
# Just showing a complete list of the words tokenized
# uncomment if you want to try
# n = 1000
# print([x[0] for x in sentence_count.most_common()[:-n-1:-1]])

In [303]:
# Working with 'Indeed Dataset'
path = os.path.join(os.getcwd(), 'indeed_dataset.csv')
df = pd.read_csv(path)

In [304]:
df.head(10)

,Title,Company,Location,Links,Salary,Description
0,Senior Software Engineer Security and Authenti...,MathWorks,"Natick, MA 01760",https://indeed.com/viewjob?jk=dedaae73a1bb2555,None,We are looking for a highly motivated work-fro...
1,None,Citizens,"Greenville, RI 02828",https://indeed.com/viewjob?jk=8b0ad10599f3a304,None,Job Description:\nAre you looking for a new an...
2,"2021745 Cyber Security Analysts $215,000.00",B4CORP,"Reston, VA",https://indeed.com/viewjob?jk=407b2677aa6ccc25,"Up to $215,000 a year",US CITIZENSHIP REQUIRED FOR THIS POSITION: Yes...
3,Senior Cyber Security Analyst,TherapyNotes.com,Remote,https://indeed.com/viewjob?jk=0dd5712bac6c57aa,None,"Cyber Security Analyst, Incident Response, Cyb..."
4,Cyber Security Analyst,NDK,"Morrisville, NC•Remote",https://indeed.com/viewjob?jk=2d39ed3a3baf0863,"Up to $130,000 a year",Information Security Engineer\nThe Information...
5,None,CentralSquare Technologies,+1 locationRemote,https://indeed.com/viewjob?jk=56f081f9afeb537e,"$85,000 - $100,000 a year","This role leads initiatives in risk, product s..."
6,Cyber Security Engineer,Softnice Inc.,Remote,https://indeed.com/viewjob?jk=8b05a98ec7ae4b42,None,Job description:\nStrong IT skills and knowled...
7,None,Amazon.com Services LLC,"New York, NY",https://indeed.com/viewjob?jk=5025373d360ec8f3,None,"Bachelor’s degree in Computer Science, Compute..."
8,Cyber Security and Networking Apprentice,United States Navy Career Center South Bay,"Redwood City, CA+2 locations",https://indeed.com/viewjob?jk=77b578b9c0642d23,"$50,000 - $95,000 a year",Your Job\nIndeed is seeking an Associate Appli...
9,None,NDK,"Morrisville, NC 27560•Remote",https://indeed.com/viewjob?jk=ed0c996377d869d3,"Up to $130,000 a year","THIS IS AN ENTRY LEVEL POSITION, NO TRAINING A..."


In [319]:
# Creating a dataframe with title as index and description as an attribute
title, description = df['Title'].to_list(), df['Description'].to_list()
indeed_df = pd.DataFrame(data={"Title": title,
                               "Description": description} 
                         )
# removing rows with 'None' for their Job-Title
indeed_df = indeed_df[indeed_df.index != 'None']

In [320]:
indeed_df.head(10)

,Title,Description
0,Senior Software Engineer Security and Authenti...,We are looking for a highly motivated work-fro...
1,None,Job Description:\nAre you looking for a new an...
2,"2021745 Cyber Security Analysts $215,000.00",US CITIZENSHIP REQUIRED FOR THIS POSITION: Yes...
3,Senior Cyber Security Analyst,"Cyber Security Analyst, Incident Response, Cyb..."
4,Cyber Security Analyst,Information Security Engineer\nThe Information...
5,None,"This role leads initiatives in risk, product s..."
6,Cyber Security Engineer,Job description:\nStrong IT skills and knowled...
7,None,"Bachelor’s degree in Computer Science, Compute..."
8,Cyber Security and Networking Apprentice,Your Job\nIndeed is seeking an Associate Appli...
9,None,"THIS IS AN ENTRY LEVEL POSITION, NO TRAINING A..."


In [348]:
# Converting the above dataframe into dictionary for iteration {title: description}
indeed = {}
for (t, d) in zip(title, description): # loops through the list title and description
    indeed[t] = d # sets the title as key and the description as the value


In [328]:
# for instance uncomment below to see how to access each description using the title:

# indeed ['Senior Software Engineer Security and Authentication']

In [365]:
# iterating through the Job's to tokenize into sentences and words
for key in indeed:
    text = indeed[key]
    # Tokenize into sentence or phrase
    sentence_count, s1 = get_sentence_counter(text=text)
    indeed_sentence = make_df(sentence_count.most_common()[:-n-1:-1], s1,
            ind_name= f'Least Occuring Sentences in\n \'{key}\'')
    # Tokenize into words
    word_count, s2 = get_text_counter(text, avoid, word_pattern)
    indeed_word = make_df(word_count.most_common()[:-n-1:-1], s2,
            ind_name= f'Least Occuring Words in\n \'{key}\'')
    break
# I am breaking here because I would like to see what could be done for just one job posting-
# then find a way to save each iteration for each job posting separately in  some format.
# For now, I am just saving the sentence/phrase and word tokens and their freqz into dataframes

In [361]:
indeed_sentence.tail(20)

,Absolute frequency,Relative frequency
Least Occuring Sentences in 'Senior Software Engineer Security and Authentication',,
Our sales staff provides knowledgeable consultative solutions leveraging a strong local and national marketing,1.0,0.011364
and then adds its own security compliance managed security and professional services nationally,1.0,0.011364
s leading security product manufacturers,1.0,0.011364
\neSecurity Solutions leverages its relationship with the industry,1.0,0.011364
paced environment and have achieved very high growth in the hot IT security industry,1.0,0.011364
We are a team of overachievers who thrive in a fast,1.0,0.011364
The ability to move and grow with our company is definitely a benefit of working at eSecurity Solutions,1.0,0.011364
\nThis is a challenging and rewarding sales opportunity with an opportunity to really make an impact,1.0,0.011364
You will inherit multiple current customer accounts to build on for your territory,1.0,0.011364


In [362]:
indeed_word.head(20)

,Absolute frequency,Relative frequency
Least Occuring Words in 'Senior Software Engineer Security and Authentication',,
fully,1.0,0.002105
availability,1.0,0.002105
preferred,1.0,0.002105
b2b,1.0,0.002105
bachelor,1.0,0.002105
education,1.0,0.002105
employees,1.0,0.002105
all,1.0,0.002105
considerations,1.0,0.002105
